# Data engineering to clean raw data and create a database

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine, text
import geopandas

# File to Load 
registration_data_to_load = Path("../Resources/chargers_by_postcode_prototype.csv")
ev_charger_data_to_load = Path("../Resources/ev_evc_prototype.csv")
fuel_type_data_to_load = Path("../Resources/NEMFUELMIX_ALL_202311022059.csv")

# Read Data Files and store into Pandas DataFrames
registration = pd.read_csv(registration_data_to_load)
ev_charger = pd.read_csv(ev_charger_data_to_load)
fuel_type = pd.read_csv(fuel_type_data_to_load)


In [2]:
# Check for missing data
ev_charger.count()

latitude            504
longitude           504
Charger type        504
Name                504
State               504
stations            504
Plug types          504
Charging Network    384
Power (kW)          320
Hardware brand      154
Address             504
Postcode            504
dtype: int64

In [3]:
# Data cleaning for ev_charger
ev_charger_fix = ev_charger.fillna(" ")
ev_charger_fix["Power (kW)"] = ev_charger_fix["Power (kW)"].replace(" ", 0)
ev_charger_fix.head()

,latitude,longitude,Charger type,Name,State,stations,Plug types,Charging Network,Power (kW),Hardware brand,Address,Postcode
0,-36.029472,146.897073,AC (level 2),Albury Waste Management Centre,NSW,1,Tesla (Type 2),Chargefox,0.0,,"565 Mudge St, Hamilton Valley NSW 2641, Australia",2641
1,-30.511849,151.669382,AC (level 2),Armidale Visitors Centre,NSW,1,Tesla (Type 2),NRMA,50.0,,"82 Marsh St, Armidale NSW 2350, Australia",2350
2,-33.906637,151.213249,AC (level 2),Audi Centre Sydney,NSW,1,Tesla (Type 2),Chargefox,0.0,,"895 S Dowling St, Zetland NSW 2017, Australia",2017
3,-28.857543,153.561096,DC (level 3),Ballina Central Shopping Centre,NSW,2,Both CHAdeMO & CCS Combo 2/SAE,Chargefox,350.0,,"56 Bangalow Rd, Ballina NSW 2478, Australia",2478
4,-34.841118,148.912191,AC (level 2),Banjo Paterson Park,NSW,1,Type 2,NRMA,50.0,,"81 Meehan St, Yass NSW 2582, Australia",2582


In [4]:
ev_charger_fix.count()

latitude            504
longitude           504
Charger type        504
Name                504
State               504
stations            504
Plug types          504
Charging Network    504
Power (kW)          504
Hardware brand      504
Address             504
Postcode            504
dtype: int64

In [5]:
ev_charger_fix.dtypes

latitude            float64
longitude           float64
Charger type         object
Name                 object
State                object
stations              int64
Plug types           object
Charging Network     object
Power (kW)          float64
Hardware brand       object
Address              object
Postcode              int64
dtype: object

In [6]:
# Data cleaning for registration
# Select wanted columns ("Top 9 ev car brands in au")
selected_registration= registration[["Postcode", "Registrations", "Year", "Audi", "BMW", "BYD", "Hyundai", "NISSAN", "Porsche", 
                                    "Renault", "TESLA", "VOLKSW"]]
selected_registration.head()

,Postcode,Registrations,Year,Audi,BMW,BYD,Hyundai,NISSAN,Porsche,Renault,TESLA,VOLKSW
0,200,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Check for missing data
selected_registration.count()

Postcode         4750
Registrations    1901
Year             4750
Audi               21
BMW               384
BYD                 9
Hyundai           238
NISSAN            337
Porsche            74
Renault            40
TESLA            1163
VOLKSW              8
dtype: int64

In [8]:
# Replace null valuues with 0
new_registration = selected_registration.fillna(0)
new_registration.head()

,Postcode,Registrations,Year,Audi,BMW,BYD,Hyundai,NISSAN,Porsche,Renault,TESLA,VOLKSW
0,200,0.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,0.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1002,0.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003,0.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1004,0.0,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
new_registration.count()

Postcode         4750
Registrations    4750
Year             4750
Audi             4750
BMW              4750
BYD              4750
Hyundai          4750
NISSAN           4750
Porsche          4750
Renault          4750
TESLA            4750
VOLKSW           4750
dtype: int64

In [10]:
new_registration.dtypes

Postcode           int64
Registrations    float64
Year               int64
Audi             float64
BMW              float64
BYD              float64
Hyundai          float64
NISSAN           float64
Porsche          float64
Renault          float64
TESLA            float64
VOLKSW           float64
dtype: object

In [11]:
# Check for missing data
fuel_type.count()

DateTime     1523
State        1523
Fuel Type    1523
Supply       1523
dtype: int64

In [12]:
fuel_type.dtypes

DateTime      object
State         object
Fuel Type     object
Supply       float64
dtype: object

In [13]:
fuel_type.head()

,DateTime,State,Fuel Type,Supply
0,29/10/2022 0:00,NSW,Battery,362.534
1,29/10/2022 0:00,NSW,Black coal,627223.514
2,29/10/2022 0:00,NSW,Gas,13468.075
3,29/10/2022 0:00,NSW,Hydro,104066.278
4,29/10/2022 0:00,NSW,Liquid Fuel,67.375


In [14]:
# Establish connection
engine = create_engine('sqlite:///Data_Engineering.db')

In [15]:
# Drop the existing table and create a new one with the desired primary key
with engine.connect() as con:
    con.execute(text('CREATE TABLE IF NOT EXISTS "ev_charger" (latitude FLOAT, longitude FLOAT, "Charger type" VARCHAR, PRIMARY KEY (latitude, longitude, "Charger type"))'))

# Output to the database
ev_charger_fix.to_sql(name='ev_charger', con=engine, if_exists='replace', index=False)

504

In [16]:
# Drop the existing table and create a new one with the desired primary key
with engine.connect() as con:
    con.execute(text('CREATE TABLE IF NOT EXISTS "fuel_type" 
                     ("DateTime" DATETIME, 
                      "State" VARCHAR, 
                      "Fuel Type" VARCHAR, 
                      "Supply" FLOAT, 
                      PRIMARY KEY ("DateTime", "State", "Fuel Type"))
                     '))

# Output to the database
fuel_type.to_sql(name='fuel_type', con=engine, if_exists='replace', index=False)

1523

In [17]:
# Drop the existing table and create a new one with the desired primary key
with engine.connect() as con:
    con.execute(text('CREATE TABLE IF NOT EXISTS "registration" 
                     ("Postcode" INTEGER, 
                      "Registrations" FLOAT, 
                      "Year" INTEGER, 
                      "Audi" FLOAT, 
                      "BMW" FLOAT, 
                      "BYD" FLOAT, 
                      "Hyundai" FLOAT, 
                      "NISSAN" FLOAT, 
                      "Porsche" FLOAT, 
                      "Renault" FLOAT, 
                      "TESLA" FLOAT, 
                      "VOLKSW" FLOAT, 
                      PRIMARY KEY ("Postcode", "Year"))
                     '))

# Output to the database
new_registration.to_sql(name='registration', con=engine, if_exists='replace', index=False)

4750

# Data engineering to merge geojson file with information of the suburbs in NSW with the location of EV chargers

In [20]:
# Importing files to merge
gdf = geopandas.read_file("../Resources/suburb-2-nsw.geojson") # geojson file
pdf = pd.read_csv('../Resources/ev_evc_prototype.csv') # CSV file

In [21]:
# Extracting name of the suburb from address column
pdf['key_column'] = pdf["Address"].str.extract(',\s*([^,]*?)\s*NSW')
pdf['key_column'] = pdf['key_column'].str.lower()
pdf.sort_values("key_column").head()

,latitude,longitude,Charger type,Name,State,stations,Plug types,Charging Network,Power (kW),Hardware brand,Address,Postcode,key_column
45,-36.083732,146.914351,AC (level 2),Rotary Kiewa St Market,NSW,1,Tesla (Type 2),NRMA,50.0,NaN,"Level 2/441 Kiewa St, Albury NSW 2640, Australia",2640,albury
151,-36.079827,146.912351,AC (level 2),Atura Hotel Albury,NSW,1,Type 2,Tesla Destination,8.0,Tesla,"Country Comfort Hotel, 648 Dean St, Albury NSW...",2640,albury
436,-32.761736,152.082891,AC (level 2),Latitude One,NSW,2,Tesla (Type 2),NaN,22.0,NaN,"Latitude Dr, Anna Bay NSW 2316, Australia",2316,anna bay
291,-30.506182,151.672433,AC (level 2),Quality Hotel Powerhouse,NSW,2,Tesla (Type 2),Tesla Destination,22.0,Tesla,"31 Marsh St, Armidale NSW 2350, Australia",2350,armidale
282,-30.554764,151.682663,AC (level 2),Petersons Armidale Winery & Guesthouse,NSW,1,J-1772,Tesla Destination,7.0,Tesla,"345 Dangarsleigh Rd, Armidale NSW 2350, Australia",2350,armidale


In [22]:
# Lowercasing values before merging them
gdf['key_column'] = gdf['nsw_loca_2'].str.lower()
gdf.sort_values("key_column").head()

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,nsw_locali,nsw_loca_1,nsw_loca_2,nsw_loca_3,nsw_loca_4,nsw_loca_5,nsw_loca_6,nsw_loca_7,geometry,key_column
1455,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.1394,26937,2016-12-12,None,NSW1,2017-02-02,None,AARONS PASS,None,None,G,None,1,"POLYGON ((149.85989 -32.83743, 149.85738 -32.8...",aarons pass
2885,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2804,27853,2017-06-13,None,NSW2,2017-08-10,None,ABBOTSBURY,None,None,G,None,1,"POLYGON ((150.86993 -33.87242, 150.86468 -33.8...",abbotsbury
191,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.192,25754,2016-02-29,None,NSW3,2016-05-08,None,ABBOTSFORD,None,None,G,None,1,"POLYGON ((151.13353 -33.84932, 151.13758 -33.8...",abbotsford
2074,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.1984,19990,2012-11-21,None,NSW4,2013-01-23,None,ABERCROMBIE,None,None,G,None,1,"POLYGON ((149.56871 -33.39590, 149.56714 -33.4...",abercrombie
960,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.1340,27104,2016-12-12,None,NSW5,2017-02-02,None,ABERCROMBIE RIVER,None,None,G,None,1,"POLYGON ((149.38332 -33.83491, 149.38492 -33.8...",abercrombie river


In [23]:
# Merged data
joined_gdf = gdf.merge(pdf, on="key_column")
joined_gdf.head()

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,nsw_locali,nsw_loca_1,nsw_loca_2,nsw_loca_3,nsw_loca_4,...,Charger type,Name,State,stations,Plug types,Charging Network,Power (kW),Hardware brand,Address,Postcode
0,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2,25065,2015-09-03,None,NSW2557,2015-11-10,None,MAYFIELD WEST,None,None,...,AC (level 2),CSIRO Energy Centre,NSW,1,J-1772,NaN,NaN,NaN,"10 Murray Dwyer Cct, Mayfield West NSW 2304, A...",2304
1,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.2,25065,2015-09-03,None,NSW2557,2015-11-10,None,MAYFIELD WEST,None,None,...,AC (level 2),CSIRO Energy Centre,NSW,1,Tesla (Type 2),NaN,NaN,NaN,"10 Murray Dwyer Cct, Mayfield West NSW 2304, A...",2304
2,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.3,25115,2015-09-03,None,NSW797,2015-11-10,None,CAMPSIE,None,None,...,AC (level 2),Canterbury Bankstown Council,NSW,2,Tesla (Type 2),NaN,NaN,NaN,"21 Shakespeare St, Campsie NSW 2194, Australia",2194
3,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.5,24884,2015-09-03,None,NSW812,2015-11-10,None,CANTERBURY,None,None,...,AC (level 2),Charles St Parking/Charging,NSW,1,Tesla (Type 2),NaN,NaN,NaN,"10B Charles St, Canterbury NSW 2193, Australia",2193
4,ckan_91e70237_d9d1_4719_a82f_e71b811154c6.8,25101,2015-09-03,None,NSW4391,2015-11-10,None,WOLLSTONECRAFT,None,None,...,DC (level 3),Nicholson St Carpark,NSW,1,Both CHAdeMO & CCS Combo 2/SAE,NaN,NaN,NaN,"19 Nicholson St, Wollstonecraft NSW 2065, Aust...",2065


In [24]:
# Exporting to a cleaned geojson
joined_gdf.to_file("Resources/final_nsw.geojson", driver="GeoJSON")